In [1]:
import pandas as pd
import requests

### Exercises
Within your codeup-data-science directory, create a new repo named time-series-exercises. This will be where you do your work for this module. Create a repository on GitHub with the same name, and link your local repository to GitHub.

Save this work in your time-series-exercises repo. Then add, commit, and push your changes.

The end result of this exercise should be a file named acquire.py.

---

<h3>Using the code from the lesson as a guide and the REST API from https://python.zach.lol/api/v1/items as we did in the lesson, create a dataframe named items that has all of the data for items.</h3>

---

In [2]:
def items():
    
    items_list = []
    url = "https://python.zach.lol/api/v1/items"

    response = requests.get(url)
    data = response.json()
    n = data['payload']['max_page']


    for i in range(1, n+1):
        new_url = url+ '?page=' + str(i)
        response = requests.get(new_url)
        data = response.json()
        page_items = data['payload']['items']
        items_list += page_items
        
    items = pd.DataFrame.from_dict(items_list)
    
    return items
    

In [3]:
items = items()

In [4]:
items.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [6]:
items.shape

(50, 6)

In [7]:
items.head(1)

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013


----
<h3>Do the same thing, but for stores (https://python.zach.lol/api/v1/stores)</h3>


---

In [8]:
def stores():
    
    stores_list = []
    url = "https://python.zach.lol/api/v1/stores"

    response = requests.get(url)
    data = response.json()
    n = data['payload']['max_page']


    for i in range(1, n+1):
        new_url = url+ '?page=' + str(i)
        response = requests.get(new_url)
        data = response.json()
        page_stores = data['payload']['stores']
        stores_list += page_stores
        
    return stores_list

In [9]:
stores = stores()

In [10]:
stores = pd.DataFrame.from_dict(stores)

stores.head(1)

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253


In [11]:
stores.shape

(10, 5)

----

<h3>Extract the data for sales (https://python.zach.lol/api/v1/sales). There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.</h3>

 - Save the data in your files to local csv files so that it will be faster to access in the future.

----

In [12]:
def sales():
    sales_list = []
    url = "https://python.zach.lol/api/v1/sales"

    response = requests.get(url)
    data = response.json()
    n = data['payload']['max_page']


    for i in range(1, n+1):
        new_url = url+ '?page=' + str(i)
        response = requests.get(new_url)
        data = response.json()
        page_sales = data['payload']['sales']
        sales_list += page_sales
        
        
        
        
    sales_df = pd.DataFrame.from_dict(sales_list)
        
    sales_df.to_csv('zachsales_df.csv')
        
    return sales_df

In [13]:
sales = pd.read_csv('zachsales_df.csv', index_col=0)

In [14]:
sales.shape

(913000, 5)

-------

<h3>Combine the data from your three separate dataframes into one large dataframe.</h3>

-----

In [15]:
sales.head()

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


In [16]:
sales.columns = ['item_id', 'sale_amount', 'sale_date', 'sale_id', 'store_id']

In [17]:
sales.head()

,item_id,sale_amount,sale_date,sale_id,store_id
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


In [18]:
items.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [19]:
sales_merge = pd.merge(items, sales, how='left', on=['item_id'], suffixes=['', '_'])
cols = ['item_id', 'item_brand', 'item_name', 'item_price', 'sale_amount', 'sale_date', 'sale_id', 'store_id', 'item_upc12', 'item_upc14']
sales_merge = sales_merge[cols]

In [20]:
sales_merge.head()

,item_id,item_brand,item_name,item_price,sale_amount,sale_date,sale_id,store_id,item_upc12,item_upc14
0,1,Riceland,Riceland American Jazmine Rice,0.84,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,35200264013,35200264013
1,1,Riceland,Riceland American Jazmine Rice,0.84,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,35200264013,35200264013
2,1,Riceland,Riceland American Jazmine Rice,0.84,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,35200264013,35200264013
3,1,Riceland,Riceland American Jazmine Rice,0.84,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,35200264013,35200264013
4,1,Riceland,Riceland American Jazmine Rice,0.84,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,35200264013,35200264013


In [21]:
stores.head(1)

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253


In [22]:
store_item_sales = pd.merge(stores, sales_merge, how='left', on=['store_id'], suffixes=['', '_'])
cols = ['item_id', 'item_brand', 'item_name', 'item_price', 'sale_amount', 'sale_date', 'sale_id', 'item_upc12', 'item_upc14', 'store_address', 
       'store_city', 'store_state', 'store_zipcode', 'store_id']
store_item_sales = store_item_sales[cols]

In [23]:
store_item_sales.head(1)

,item_id,item_brand,item_name,item_price,sale_amount,sale_date,sale_id,item_upc12,item_upc14,store_address,store_city,store_state,store_zipcode,store_id
0,1,Riceland,Riceland American Jazmine Rice,0.84,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,1


In [24]:
store_item_sales.shape

(913000, 14)

----

<h3>Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here:</h3>

----

In [25]:
wind_energy = pd.read_csv('opsd_germany_daily.csv', index_col= 0)

wind_energy.head()

,Consumption,Wind,Solar,Wind+Solar
Date,,,,
2006-01-01,1069.184,NaN,NaN,NaN
2006-01-02,1380.521,NaN,NaN,NaN
2006-01-03,1442.533,NaN,NaN,NaN
2006-01-04,1457.217,NaN,NaN,NaN
2006-01-05,1477.131,NaN,NaN,NaN


In [26]:
def missing_zero_values_table(df):
    
    '''This function will look at any data set and report back on zeros and nulls for every column while also giving percentages of total values
        and also the data types. The message prints out the shape of the data frame and also tells you how many columns have nulls '''
    
    
    
    zero_val = (df == 0.00).astype(int).sum(axis=0)
    null_count = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mz_table = pd.concat([zero_val, null_count, mis_val_percent], axis=1)
    mz_table = mz_table.rename(
    columns = {0 : 'Zero Values', 1 : 'null_count', 2 : '% of Total Values'})
    mz_table['Total Zeroes + Null Values'] = mz_table['Zero Values'] + mz_table['null_count']
    mz_table['% Total Zero + Null Values'] = 100 * mz_table['Total Zeroes + Null Values'] / len(df)
    mz_table['Data Type'] = df.dtypes
    mz_table = mz_table[
        mz_table.iloc[:,1] >= 0].sort_values(
        '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " +  str((mz_table['null_count'] != 0).sum()) +
          " columns that have NULL values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)

    return mz_table


In [27]:
missing_zero_values_table(wind_energy)

Your selected dataframe has 4 columns and 4383 Rows.
There are 3 columns that have NULL values.


,Zero Values,null_count,% of Total Values,Total Zeroes + Null Values,% Total Zero + Null Values,Data Type
Wind+Solar,0,2196,50.1,2196,50.1,float64
Solar,0,2195,50.1,2195,50.1,float64
Wind,0,1463,33.4,1463,33.4,float64
Consumption,0,0,0.0,0,0.0,float64
